# Fundamentals of Software Systems (FSS)
**Software Evolution – Part 02 Assignment**

## Submission Guidelines

To correctly complete this assignment you must:
* Carry out the assignment in a team of 2 to 4 students.
* Carry out the assignment with your team only. You are allowed to discuss solutions with other teams, but each team should come up its own personal solution. A strict plagiarism policy is going to be applied to all the artifacts submitted for evaluation.
* As your submission, upload the filled Jupyter Notebook (including outputs) together with the d3 visualization web pages (i.e. upload everything you downloaded including the filled Jupyter Notebook plus your `output.json`)
* The files must be uploaded to OLAT as a single ZIP (`.zip`) file by Dec 12, 2022 @ 23:55.


## Group Members
* Firstname, Lastname, Immatrikulation Number
* Lucius, Bachmann, 11-060-274
* Adnreas, Wiemeyer, 15-728-405

## Task Context

In this assigment we will be analyzing the _elasticsearch_ project. All following tasks should be done with the subset of commits from tag `v1.0.0` to tag `v1.1.0`.

Website: https://github.com/elastic/elasticsearch
Repository: https://github.com/elastic/elasticsearch.git

In [181]:
from enum import Enum 

class Modification(Enum):
    ADDED = "Lines added"
    REMOVED = "Lines removed"
    TOTAL = "Lines added + lines removed"
    DIFF = "Lines added - lines removed"

In [182]:
# noinspection PyUnresolvedReferences
import os.path
from datetime import datetime
from os import path, mkdir

# noinspection PyUnresolvedReferences
import pandas as pd
# noinspection PyUnresolvedReferences
import plotly.express as px
from pydriller import Repository, Git

repo_remote_path = 'https://github.com/elastic/elasticsearch.git'
repo_owner = 'elastic'
repo_name = 'elasticsearch'
repo_checkout_path = f'{repo_owner}/{repo_name}'

if not path.exists(repo_owner):
    mkdir(repo_owner)

from_tag = 'v1.0.0'
to_tag = 'v1.1.0'
repo = Repository(repo_remote_path, clone_repo_to=repo_owner, from_tag=from_tag, to_tag=to_tag)
# clone repo if necessary
for commit in repo.traverse_commits():
    break
git = Git(repo_checkout_path)

## Task 1: Author analysis

In the following, please consider only `java` files.

The first task is to get an overview of the author ownership of the _elasticsearch_ project. In particular, we want to understand who are the main authors in the system between the two considered tags, the authors distribution among files and the files distribution among authors. To this aim, perform the following:
- create a dictionary (or a list of tuples) with the pairs author => number of modified files
- create a dictionary (or a list of tuples) with the pairs file => number of authors who modified the file
- visualize the distribution of authors among files: the visualization should have on the x axis the number of authors per file (from 1 to max), and on the y axis the number of files with the given number of authors (so for example the first bar represent the number of files with single author)
- visualize the distribution of files among authors: the visualization should have on the x axis the number of files per author (from 1 to max), and on the y axis the number of authors who modified the given number of files (so for example the first bar represent the minor contributors, i.e., the number of authors who changed only 1 file)

Comment the two distribution visualizations.



Now, let's look at the following 3 packages in more detail:
1. `src/main/java/org/elasticsearch/search`
2. `src/main/java/org/elasticsearch/index`
3. `src/main/java/org/elasticsearch/action`

Create a function that, given the path of a package and a modification type (see class Modification above), returns a dictionary of authors => number, where the number counts the total lines added or removed or added+removed or added-removed (depending on the given Modification parameter), for the given package. To compute the value at the package level, you should aggregate the data per file.

Using the function defined above, visualize the author contributions (lines added + lines removed). The visualization should have the author on the x axis, and the total lines on the y axis. Sort the visualization in decreasing amount of contributions, i.e., the main author should be the first.

Compare the visualization for the 3 packages and comment.

Count the number of authors per file and the number of files per author.

In [183]:
from pandas import DataFrame
from typing import Set

repo = Repository(repo_checkout_path, from_tag=from_tag, to_tag=to_tag)

files_to_authors : {str, Set[str]} = {}
authors_to_files : {str, Set[str]} = {}

for commit in repo.traverse_commits():
    author_identifier = commit.author.name
    authors_to_files.setdefault(author_identifier, set())
    for file in commit.modified_files:
        if file.new_path is not None:
            files_to_authors.setdefault(file.new_path, set())
            files_to_authors[file.new_path].add(author_identifier)
            authors_to_files[author_identifier].add(file.new_path)

Visualize Nr of authors per file:

In [184]:
file_author_count = {(k, len(authors)) for k, authors in files_to_authors.items()}
col_name_author = 'File'
col_name_nr_of_authors = 'Nr of authors'
df_file_author_count = DataFrame(file_author_count, columns=[col_name_author, col_name_nr_of_authors])
df_file_author_count_sorted = df_file_author_count.sort_values(by=[col_name_nr_of_authors], ascending=False)

The 20 files with the most authors:

In [185]:
display(df_file_author_count_sorted[:20])

,File,Nr of authors
205,pom.xml,13
989,src/test/java/org/elasticsearch/test/Elasticse...,8
60,src/test/java/org/elasticsearch/test/TestClust...,7
98,src/main/java/org/elasticsearch/percolator/Per...,6
411,src/test/java/org/elasticsearch/index/query/Si...,6
89,src/main/java/org/elasticsearch/index/engine/i...,5
1118,src/main/java/org/elasticsearch/rest/action/Re...,5
476,src/main/java/org/elasticsearch/search/SearchS...,5
207,src/main/java/org/elasticsearch/search/interna...,5
1255,docs/reference/mapping/types/core-types.asciidoc,5


Distribution of author count per file:

In [186]:
hist_author_count = px.histogram(df_file_author_count_sorted, x=col_name_nr_of_authors)
hist_author_count.show()
hist_author_log_count = px.histogram(df_file_author_count_sorted, x=col_name_nr_of_authors, log_y=True)
hist_author_log_count.show()

As we can see in a lot of projects, there are certain files which are touched by a lot of authors, but most files are touched by a single author.
The file with the most authors is the pom.xml, where the project configuration is stored.
There every developer has to change the file if e.g. a dependency is added, removed or updated.
Then follow some tests with 8 or 7 coauthors, and then we are already at a moderate 7, but only between 2 minor releases.

Visualise Nr of files per author:

In [187]:
author_file_count = {(k, len(files)) for k, files in authors_to_files.items()}
col_name_author = 'Author'
col_name_nr_of_files = 'Nr of files'
df_author_file_count = DataFrame(author_file_count, columns=[col_name_author, col_name_nr_of_files])
df_author_file_count_sorted = df_author_file_count.sort_values(by=[col_name_nr_of_files], ascending=False)

The 20 authors which changed the most files:

In [188]:
display(df_author_file_count_sorted[:20])

,Author,Nr of files
52,javanna,191
42,Martijn van Groningen,186
9,Adrien Grand,174
0,Simon Willnauer,155
10,Clinton Gormley,129
19,uboness,113
63,Luca Cavanna,105
20,Lee Hinman,76
43,Shay Banon,67
35,David Pilato,63


Distribution of files modified per author:

In [189]:
hist_author_count = px.histogram(df_author_file_count_sorted, x=col_name_nr_of_files)
hist_author_count.show()

Most of the people only change a few files, as has been seen in other open source projects.
There are some main contributes which changed more than 20 files.


Now, let's look at the following 3 packages in more detail:
1. `src/main/java/org/elasticsearch/search`
2. `src/main/java/org/elasticsearch/index`
3. `src/main/java/org/elasticsearch/action`

Create a function that, given the path of a package and a modification type (see class Modification above), returns a dictionary of authors => number, where the number counts the total lines added or removed or added+removed or added-removed (depending on the given Modification parameter), for the given package. To compute the value at the package level, you should aggregate the data per file.

Using the function defined above, visualize the author contributions (lines added + lines removed). The visualization should have the author on the x axis, and the total lines on the y axis. Sort the visualization in decreasing amount of contributions, i.e., the main author should be the first.

Compare the visualization for the 3 packages and comment.

In [190]:
from typing import Dict


AuthorChurnDict = Dict[str, Dict[Modification, int]]

# noinspection PyShadowingNames
def author_churn_of_path(path: str, from_tag: str = from_tag, to_tag: str = to_tag) -> AuthorChurnDict:
    modifications: AuthorChurnDict = {}
    repo = Repository(path_to_repo=repo_checkout_path, from_tag=from_tag, to_tag=to_tag)
    for commit in repo.traverse_commits():
        author_identifier = commit.author.name
        modifications.setdefault(author_identifier, dict([(mod_type, 0) for mod_type in Modification]))
        for file in commit.modified_files:
            if file.new_path is None:
                continue
            if not file.new_path.__contains__(path):
                continue
            modifications[author_identifier][Modification.ADDED] += file.added_lines
            modifications[author_identifier][Modification.REMOVED] += file.deleted_lines
            modifications[author_identifier][Modification.TOTAL] += file.added_lines + file.deleted_lines
            modifications[author_identifier][Modification.DIFF] += file.added_lines - file.deleted_lines

    return modifications

display([(k, v) for k, v in author_churn_of_path('src/main/java/org/elasticsearch/search').items()][:20])

[('Simon Willnauer',
  {<Modification.ADDED: 'Lines added'>: 93,
   <Modification.REMOVED: 'Lines removed'>: 202,
   <Modification.TOTAL: 'Lines added + lines removed'>: 295,
   <Modification.DIFF: 'Lines added - lines removed'>: -109}),
 ('Lee Hinman',
  {<Modification.ADDED: 'Lines added'>: 244,
   <Modification.REMOVED: 'Lines removed'>: 25,
   <Modification.TOTAL: 'Lines added + lines removed'>: 269,
   <Modification.DIFF: 'Lines added - lines removed'>: 219}),
 ('Adrien Grand',
  {<Modification.ADDED: 'Lines added'>: 1823,
   <Modification.REMOVED: 'Lines removed'>: 370,
   <Modification.TOTAL: 'Lines added + lines removed'>: 2193,
   <Modification.DIFF: 'Lines added - lines removed'>: 1453}),
 ('Martijn van Groningen',
  {<Modification.ADDED: 'Lines added'>: 15,
   <Modification.REMOVED: 'Lines removed'>: 67,
   <Modification.TOTAL: 'Lines added + lines removed'>: 82,
   <Modification.DIFF: 'Lines added - lines removed'>: -52}),
 ('Clinton Gormley',
  {<Modification.ADDED: 'Lines

## Task 2: Knowledge loss

We now want to analyze the knowledge loss when the main contributor of the analyzed project would leave. For this we will use the circle packaging layout introduced in the "Code as a Crime Scene" book. It should show how much of each file was written by the main contributor of _elasticsearch_ (according to the analysis above using `Modification.TOTAL`) and indicate which areas would be affected most when this contributor leaves the project. This assignment includes the necessary `knowledge_loss.html` file as well as the `d3` folder with the d3 dependencies. Your task is to create the `output.json` file according to the specification below. This file can then be visualized with the files provided.

For showing the visualization, once you have the output as `output.json` you should
* make sure to have the `knowledge_loss.html` file in the same folder
* start a local HTTP server in the same folder (e.g. with python `python3 -m http.server`, serving necessary for d3)
* open the served `knowledge_loss.html` and look at the visualization

For testing, you can use the provided `output.json` and should see a circle packaging layout with two circles, one big red, and one small white-red.

For the package you identify as the worst in terms of knowledge loss, investigate the author contributions using the function defined in the previous exercise and comment how the situation is, e.g. how big the gap between the main author and the second biggest contributor for the selected package is.

### Output Format for Visualization

Example:

* `root` is always the root of the tree
* `size` should be the total number of lines of contribution
* `weight` can be set to the same as `size`
* `ownership` should be set to the percentage of contributions from the main author (e.g. 0.98 for 98% if contributions coming from the main author)

```
{
  "name": "root",
  "children": [
    {
      "name": "test",
      "children": [
        {
          "name": "benchmarking",
          "children": [
            {
              "author_color": "red",
              "size": "4005",
              "name": "t6726-patmat-analysis.scala",
              "weight": 1.0,
              "ownership": 0.9,
              "children": []
            },
            {
              "author_color": "red",
              "size": "55",
              "name": "TreeSetIterator.scala",
              "weight": 0.88,
              "ownership": 0.2,
              "children": []
            }
          ]
        }
      ]
    }
  ]
}
```

### JSON Export

For exporting the data to JSON you can use the following snippet:

```
import json

with open("output.json", "w") as file:
    json.dump(tree, file, indent=4)
```

## Task 3: Code Churn Analysis

The third and last task is to analyze the code churn of the _elasticsearch_ project. For this analysis we look at the code churn, meaning the daily change in the total number of lines of the project. Visualize the code churn over time bucketing the data by day. Remember that you'll need to fill the gaps for days when there are no commits. Chose a filling strategy and justify it.

Look at the churn trend over time and identify two outliers. For each of them:
- identify if it was caused by a single or multiple commits (since you are bucketing the data by day)
- find the hash of the involved commit(s)
- find the involved files
- look at the actual diff

Based on the above, discuss if the outlier is a false positive or should be a reason for concern.

In [191]:
date_to_churn = {}

for commit in repo.traverse_commits():
    date = commit.committer_date.date()
    if date in date_to_churn:
        date_to_churn[date]["lines added"] += commit.insertions
        date_to_churn[date]["lines deleted"] += commit.deletions
        date_to_churn[date]["hashes"].append(commit.hash)
        date_to_churn[date]["messages"].append(commit.msg)
    else:
        date_to_churn[date] = {"lines added":commit.insertions,
                               "lines deleted":commit.deletions,
                               "hashes":[commit.hash],
                               "messages":[commit.msg]}



I choose to add zeros to fill the gaps, because if there are no commits, it means that the amount of lines could not have changed.

In [192]:
min_date = min(date_to_churn.keys())
max_date = max(date_to_churn.keys())
date_list = [entry.date() for entry in pd.date_range(min_date, max_date)]
for date in date_list:
    if date not in date_to_churn:
        date_to_churn[date] = {"lines added":0,
                               "lines deleted":0,
                               "hashes":[],
                               "messages":[]}

In [193]:
from bokeh.io.output import output_notebook
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, RangeTool
from bokeh.models.tools import HoverTool
from bokeh.plotting import figure, show
import numpy as np

output_notebook()

items = list(date_to_churn.items())
items.sort(key=lambda t: t[0])
dates = [datetime.combine(item[0], datetime.min.time()) for item in items]

source = ColumnDataSource(data=dict(
    date=dates,
    added=[item[1]['lines added'] for item in items],
    deleted=[item[1]['lines deleted'] for item in items],
    hashes=[len(item[1]['hashes']) for item in items],#[item[1]['hashes'] for item in items], TODO
    ))

p = figure(height=300, width=800, tools="xpan", toolbar_location=None,
           x_axis_type="datetime", x_axis_location="above",
           x_range=(dates[0], dates[int(len(dates)/4)]))

p.line('date', 'added', source=source, color='green', legend_label='added lines')
p.line('date', 'deleted', source=source, color='red', legend_label='deleted lines')
p.yaxis.axis_label = 'lines of change'
p.legend.location = "top_right"
p.add_tools(HoverTool(tooltips=[
            ('date', '@date{%F}'),
            ('added', '@added'),
            ('deleted', '@deleted'),
            ('amount of commits', '@hashes')
            ],
            formatters={
            '@date'        : 'datetime',
            #'@hashes'      : 'printf',# use 'datetime' formatter for '@date' field
            }))

select = figure(height=130, width=800, y_range=p.y_range,
                x_axis_type="datetime", y_axis_type=None,
                tools="", toolbar_location=None)

range_tool = RangeTool(x_range=p.x_range)

select.line('date', 'added', source=source, color='green')
select.line('date', 'deleted', source=source, color='red')

select.add_tools(range_tool)
select.toolbar.active_multi = range_tool

show(column(p, select))

Loading BokehJS ...

In [198]:
def commits_on(date):
    hashes = date_to_churn[date]['hashes']

    commits = []
    for commit in repo.traverse_commits():
        if commit.hash in hashes:
            commits.append(commit)
    return commits

def date_overview(date):
    print(date)
    for commit in commits_on(date):
        print_commit_overview(commit)

def print_commit_overview(commit):
    print("\nhash:", commit.hash)
    print("msg:", commit.msg)
    for file in commit.modified_files:
        print(f"+{file.added_lines}   \t-{file.deleted_lines}   \t{file.new_path if file.new_path else 'del '+file.old_path}")

def print_diff(diff):
    for code_line in diff.split('\n'):
        print(code_line)

In [195]:
analysed_dates_tuples = [(2014, 3, 13), (2014, 2, 26)]
analysed_dates = [datetime(*tuple).date() for tuple in analysed_dates_tuples]

In [199]:
date_overview(analysed_dates[0])

2014-03-13

hash: a7230f8d3aa128bd3de112a619e05b191f4df320
msg: Disable by default plugins isolation
+1   	-1   	src/main/java/org/elasticsearch/plugins/PluginsService.java

hash: e99c666f8226eebb6daeb7ae1839e28babb38d64
msg: [TESTS] Fix incorrect discovery options.
+2   	-2   	src/test/java/org/elasticsearch/discovery/DiscoveryTests.java

hash: 822f0873dcb7ee8b678c23c041d6a6966851d830
msg: [DOCS] Added spring data elasticsearch integration
+3   	-0   	docs/community/integrations.asciidoc

hash: f7cd6d43aad4a250bd1f3be199c62cfd6b927e91
msg: [DOCS] Various aggregation doc fixes
+21   	-21   	docs/reference/search/aggregations.asciidoc

hash: c7a3781dcdebe459a6964d669e92a75c4b480587
msg: For unicast zen discovery don't overwrite a ping response for a node if the previous ping response has a set master and the current response hasn't.

Per single main ping request we maintain the received ping response per node. Each node level ping response is mapped into that. If from a previous node le

The first code churn outlier on 2014-02-26 contains a huge amount of changes, which are non-trivial. So it is indeed a problematic outlier.

In [200]:
date_overview(analysed_dates[1])

2014-02-26

hash: c88aa24bacfbde6e0f65a7f4a45bdab041d96795
msg: Refactored TransportIndexReplicationOperationAction to be able to expose the shard id related to a shard failure

The `ShardOperationFailedException` is now created within `TransportIndexReplicationAction` passing in the current shard id as a constructor argument.
Also replaced `AtomicReferenceArray<Object>` with `AtomicReferenceArray<ShardActionResult>`, where `ShardActionResult` wraps the `ShardResponse` or the failure, containing all the needed info.
+2   	-2   	src/main/java/org/elasticsearch/action/delete/index/IndexDeleteResponse.java
+4   	-15   	src/main/java/org/elasticsearch/action/delete/index/TransportIndexDeleteAction.java
+2   	-17   	src/main/java/org/elasticsearch/action/deletebyquery/TransportIndexDeleteByQueryAction.java
+56   	-8   	src/main/java/org/elasticsearch/action/support/replication/TransportIndexReplicationOperationAction.java
+13   	-5   	src/test/java/org/elasticsearch/deleteByQuery/DeleteByQu

The second code churn outlier on 2014-02-26 also contains a huge amount of changes, which are non-trivial. So it is indeed a problematic outlier.